In [13]:
from google.colab import drive
drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/CART498/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import logging
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2").to("cuda")
model.eval()

# Suppress informational messages and warnings
logging.getLogger("transformers").setLevel(logging.ERROR)

In [15]:
def generate_oulipian_poem(poem, x, seed):
    torch.manual_seed(seed)

    new_poem = ""

    for line in poem.strip().split("\n"):
        # Split the line into everything before the last word
        before, last_token = line.rsplit(" ", 1)

        # Extract the word and punctuation from the last token
        match = re.match(r"(.+?)([^\w\s]*)$", last_token)
        last_word, punctuation = match.groups()

        # Create prompt and get model predictions
        prompt = before + " " + last_word
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

        with torch.no_grad():
            logits = model(input_ids).logits[0, -1]

        probs = torch.softmax(logits, dim=-1)
        # Get the top x predictions to ensure we have enough
        top_probs, top_indices = torch.topk(probs, k=min(x + 5, len(probs)))

        # Select the x-th ranked token
        if x - 1 < len(top_indices):
            predicted_token = top_indices[x - 1]
        else:
            predicted_token = top_indices[-1]  # Fallback to last available

        predicted_word = tokenizer.decode([predicted_token]).strip()

        new_poem += before + " " + predicted_word + punctuation + "\n"

    return new_poem



In [16]:
poem = """
One must have a mind of winter
To regard the frost and the boughs
Of the pine-trees crusted with snow;
And have been cold a long time
To behold the junipers shagged with ice,
The spruces rough in the distant glitter
Of the January sun; and not to think
Of any misery in the sound of the wind,
In the sound of a few leaves,
Which is the sound of the land
Full of the same wind
That is blowing in the same bare place
For the listener, who listens in the snow,
And, nothing himself, beholds
Nothing that is not there and the nothing that is.
"""

In [19]:
# Generate P+7 version
prediction_level = 7
oulipian_poem = generate_oulipian_poem(poem, prediction_level, seed=0)
with open(f'{drive_path}P+7.txt', 'w') as f:
    f.write(oulipian_poem)
print(oulipian_poem)

One must have a mind of that
To regard the frost and the and
Of the pine-trees crusted with that;
And have been cold a long in
To behold the junipers shagged with as,
The spruces rough in the distant -
Of the January sun; and not to I
Of any misery in the sound of the that,
In the sound of a few coming,
Which is the sound of the in
Full of the same as
That is blowing in the same bare ,"
For the listener, who listens in the ?,
And, nothing himself, what
Nothing that is not there and the nothing that here.



In [20]:
# Generate P+23 version
prediction_level = 23
oulipian_poem_px = generate_oulipian_poem(poem, prediction_level, seed=0)
with open(f'{drive_path}P+{prediction_level}.txt', 'w') as f:
    f.write(oulipian_poem_px)
print(oulipian_poem_px)

One must have a mind of on
To regard the frost and the it
Of the pine-trees crusted with m;
And have been cold a long …
To behold the junipers shagged with while,
The spruces rough in the distant ings
Of the January sun; and not to but
Of any misery in the sound of the :,
In the sound of a few hitting,
Which is the sound of the crashing
Full of the same is
That is blowing in the same bare we
For the listener, who listens in the outside,
And, nothing himself, you
Nothing that is not there and the nothing that on.

